# 03. Experiment - Synthetic Data Evaluation (Stratified 5x2 CV)

This notebook performs a scientifically rigorous evaluation of Random Forest, CatBoost, and a Stacking Ensemble across various synthetic datasets. 

We use **Stratified 5x2 Cross-Validation** (Dietterich's rule) to ensure statistical stability and handle class imbalance in the `mayo` score.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, make_scorer, confusion_matrix
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_theme(style="whitegrid")

## 1. Run Iteration over All Datasets

We iterate through all imputation (directories) and synthesis (files) methods, applying Stratified 5x2 CV to each combination.

In [ ]:
synthetic_root = '../data/synthetic'
all_results = []
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=42)

for imputation_method in os.listdir(synthetic_root):
    imputation_path = os.path.join(synthetic_root, imputation_method)
    if not os.path.isdir(imputation_path): continue
    
    for file in os.listdir(imputation_path):
        if not file.endswith('.csv'): continue
        
        synthesis_method = file.replace('uc_diagnostics_', '').replace('.csv', '')
        file_path = os.path.join(imputation_path, file)
        
        # Load data
        df = pd.read_csv(file_path)
        
        # Prepare features and target
        X = df.drop(columns=['mayo'])
        y = df['mayo']
        
        print(f"Processing: Imputation={imputation_method}, Synthesis={synthesis_method}...")
        
        # Define models
        models = {
            'RF': RandomForestClassifier(n_estimators=100, random_state=42),
            'CatBoost': CatBoostClassifier(iterations=500, random_seed=42, verbose=0),
            'Stacking': StackingClassifier(
                estimators=[
                    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
                    ('cb', CatBoostClassifier(iterations=500, random_seed=42, verbose=0))
                ], 
                final_estimator=LogisticRegression()
            )
        }
        
        scoring = {
            'balanced_accuracy': make_scorer(balanced_accuracy_score),
            'accuracy': make_scorer(accuracy_score),
            'f1_weighted': make_scorer(f1_score, average='weighted')
        }
        
        for model_name, model in models.items():
            # Perform 5x2 CV
            cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring, n_jobs=-1)
            
            # Store all 10 results for each fold/repeat
            for i in range(10):
                row = {
                    'imputation': imputation_method,
                    'synthesis': synthesis_method,
                    'model': model_name,
                    'fold_id': i,
                    'balanced_accuracy': cv_results['test_balanced_accuracy'][i],
                    'accuracy': cv_results['test_accuracy'][i],
                    'f1_weighted': cv_results['test_f1_weighted'][i]
                }
                all_results.append(row)

results_df = pd.DataFrame(all_results)
print("Done!")

## 2. Results Visualization (Distributions)

We use boxplots to show the distribution of performance across the 10 folds for each configuration.

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(data=results_df, x='imputation', y='balanced_accuracy', hue='model')
plt.title('Balanced Accuracy Distribution by Imputation Method (5x2 CV)')
plt.ylabel('Balanced Accuracy')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(data=results_df, x='synthesis', y='balanced_accuracy', hue='model')
plt.title('Balanced Accuracy Distribution by Synthesis Method (5x2 CV)')
plt.ylabel('Balanced Accuracy')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

## 3. Statistical Summary and Best Configuration

In [ ]:
# Group by config and calculate mean/std
summary_df = results_df.groupby(['imputation', 'synthesis', 'model']).agg({
    'balanced_accuracy': ['mean', 'std'],
    'accuracy': 'mean',
    'f1_weighted': 'mean'
}).reset_index()

# Flatten multi-index columns
summary_df.columns = ['imputation', 'synthesis', 'model', 'mean_balanced_accuracy', 'std_balanced_accuracy', 'mean_accuracy', 'mean_f1_weighted']

best_row = summary_df.sort_values(by='mean_balanced_accuracy', ascending=False).iloc[0]
print("Best Performing Configuration (Based on Mean Balanced Accuracy):")
print(best_row)

print("\nTop 5 Overall:")
display(summary_df.sort_values(by='mean_balanced_accuracy', ascending=False).head(5))

## 4. Performance Heatmap (Stacking Model)

Visualizing the interaction between imputation and synthesis for the Stacking model.

In [ ]:
stacking_results = summary_df[summary_df['model'] == 'Stacking']
matrix = stacking_results.pivot(index='imputation', columns='synthesis', values='mean_balanced_accuracy')

plt.figure(figsize=(12, 8))
sns.heatmap(matrix, annot=True, cmap='YlGnBu', fmt='.3f')
plt.title('Stacking Model - Mean Balanced Accuracy (5x2 CV)')
plt.show()

## 5. Best Configuration: Confusion Matrix

Generating an aggregated confusion matrix for the best overall configuration found above using the manual CV loop.

In [ ]:
best_imp = best_row['imputation']
best_syn = best_row['synthesis']
best_model_name = best_row['model']

print(f"Generating Confusion Matrix for Best Configuration: {best_imp} + {best_syn} with {best_model_name}")

# Load the best dataset
best_file_path = f"../data/synthetic/{best_imp}/uc_diagnostics_{best_syn}.csv"
df_best = pd.read_csv(best_file_path)
X_best = df_best.drop(columns=['mayo'])
y_best = df_best['mayo']

# Define the best model type
if best_model_name == 'RF':
    model_best = RandomForestClassifier(n_estimators=100, random_state=42)
elif best_model_name == 'CatBoost':
    model_best = CatBoostClassifier(iterations=500, random_seed=42, verbose=0)
else: # Stacking
    model_best = StackingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('cb', CatBoostClassifier(iterations=500, random_seed=42, verbose=0))
        ], 
        final_estimator=LogisticRegression()
    )

# Manual aggregation over 5x2 folds since cross_val_predict doesn't support repeats
labels = np.unique(y_best)
cm_total = np.zeros((len(labels), len(labels)), dtype=int)

for train_index, test_index in cv.split(X_best, y_best):
    X_train_fold, X_test_fold = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train_fold, y_test_fold = y_best.iloc[train_index], y_best.iloc[test_index]
    
    model_best.fit(X_train_fold, y_train_fold)
    y_pred_fold = model_best.predict(X_test_fold)
    
    cm_total += confusion_matrix(y_test_fold, y_pred_fold, labels=labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_total, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix (Aggregated 5x2 CV)\n{best_model_name} on {best_imp}+{best_syn}')
plt.show()